# 1. Giới thiệu:
MobileNet là một họ kiến trúc CNN được phát triển bởi Google, với mục tiêu tối ưu hóa cho các thiết bị di động và hệ thống có tài nguyên hạn chế như điện thoại thông minh, thiết bị IoT hay vi xử lý nhúng. Ý tưởng là tách các kênh trong 1 ảnh ra, học riêng các feature trong từng kênh, sau đó mới gộp lại. Các mô hình trong họ MobileNet giúp giảm thiểu số lượng parameters và lượng phép tính (FLOPs), đồng thời vẫn duy trì độ chính xác cao trong các bài toán thị giác máy tính.

Các ứng dụng phổ biến:
- Phân loại hình ảnh
- Nhận diện đối tượng
- Nhận diện khuôn mặt
- Phát hiện tư thế
- Sử dụng trong các framework như: TensorFlow Lite, CoreML, ONNX,...

Dòng này tốt nhất là 128 x 128 -> 224 x 224 (do ImageNet)

# 2. Vấn đề:
Để dễ mô tả trong các phần dưới, ta quy ước:
- M: số kênh đầu vào
- N: số kênh đầu ra
- K: kích thước kernel (thường là 3)
- H × W: kích thước không gian đầu vào

Trong CNN truyền thống, một lớp convolution chuẩn sử dụng một tập kernel để trích xuất đặc trưng từ toàn bộ ảnh đầu vào, đồng thời trộn thông tin giữa các kênh màu để tạo ra một đặc trưng tổng hợp.

Cách hoạt động: Tạo ra N filter 3D K * K * M. Mỗi filter này trực tiếp duyệt qua hết ảnh ban đầu, vừa học feature xuyên suốt các kênh vừa kết hợp các kênh tạo ra 1 kênh 2D. Việc lặp lại N filter sẽ thu được N kênh.

Cách hoạt động của nó gây ra 3 vấn đề chính:

### 1. FLOPs:
FLOPs cho 1 lớp convolution truyền thống:
$$FLOPs=N * [(H*W) * (K*K*M)] = H * W * K^2 * M * N$$

Điều này rất tốn kém, đặc biệt khi số kênh lớn. Lấy ví dụ 1 ảnh đầu vào 224x224x3, N = 64, K = 3:

$$FLOPs=224*224*3^2*3*64 = 86704128$$

### 2. Parameters:
Số parameter cũng bị ảnh hưởng nghiêm trọng. Ta có công thức:
$$\text{layer param} = (K*K*M)*N = K^2 * M * N$$

Ví dụ với $K=3, M=128, N=256$, ta đã có:
$$\text{layer param} = 3^2*128*256 = 294912$$

Đây là số lượng tham số chỉ cho 1 lớp CNN. Vấn đề này khiến bộ nhớ và thời gian tải mô hình tăng mạnh.

### 3. Ép 1 bộ lọc làm 2 nhiệm vụ cùng lúc:
Khi ta áp dụng cùng 1 kernel cho toàn bộ không gian ảnh, bộ lọc phải làm cùng lúc 2 việc:
- Học cách trích xuất các feature không gian (edge, texture,...)
- Học cách kết hợp thông tin giữa các kênh màu

Việc này khi tách ra sẽ giảm rất nhiều độ phức tạp tính toán.


# 3. Giải pháp đề xuất:
### 1. Intuition đằng sau giải pháp: 
Không phải lớp conv nào cũng cần học quan hệ giữa các kênh cùng 1 lúc. Các feature có thể được trích xuất trong những kênh riêng biệt, rồi mới kết hợp lại như trong xử lý ảnh cổ điển (dùng kernel như Sobel để lọc riêng từng kênh, sau đó mới gộp thông tin)
### 2. Depthwise seperable convolution (gọi là DSC cho nhanh):
Note: Từ seperable này dùng để chỉ việc tách 2 chức năng riêng biệt trong khối chứ không phải là tách từng kênh để xử lý, vì vốn dĩ depthwise convolution đã tách ra riêng từng filter cho từng kênh rồi.
Về cơ bản, conv truyền thống gộp hai nhiệm vụ lọc không gian và kết hợp kênh vào một bước duy nhất. Trong khi đó, DSC phân tách hai nhiệm vụ này ra và sử dụng 2 phép conv để thực hiện thay vì 1.
Thay vì dùng convolution 3D thông thường, ta thay bằng 2 bước riêng biệt:
- Depthwise convolution (lọc không gian - spatial filtering):
    - M filter 2D (thường là $3*3$) được áp dụng tương ứng với M kênh đầu vào, mỗi filter sẽ đảm nhận xử lý 1 kênh.
    - Output sẽ là M kênh, giờ đây có chứa những thông tin về spatial feature.
    - Intuition kiểu như bạn cho N đứa trẻ tô đậm những phần khác nhau của 1 hình ảnh, thì đầu ra là M ảnh chứa những chi tiết tô đậm riêng biệt.
    
    VD: ảnh inp có 3 kênh màu -> tensor vẫn có 3 kênh nhờ áp dụng kernel $3*3$ cho mỗi kênh input.
- Pointwise convolution (kết hợp kênh - channel aggregation):
    - Đây là giai đoạn quyết định số kênh output, bằng cách dùng N filter 3D $1*1*M$ để kết hợp các giá trị tại cùng vị trí không gian nhưng trên nhiều kênh khác nhau.
    - Đây là điểm khác biệt lớn nhất của khối này so với phép truyền thống, vì hoàn toàn không học thêm bất cứ gì về quan hệ không gian, chỉ gộp kênh thôi (kiểu nén kênh ấy).

    VD: 3 kênh đầu vào (sau deptwise conv) -> 64 kênh output bằng cách dùng 64 kernel $1*1*3$

Ta có thể xem ảnh minh họa để hiểu hơn về khối:
<img src="image0.png"> 
Như vậy, số FLOPs sử dụng:
$$FLOPs = (H*W)*(K*K*M) + H*W*(N*1*1*M) = H*W*M(K^2+N)$$
Lại lấy ví dụ ảnh đầu vào 224x224x3, N = 64, K = 3:
$$FLOPs = 224*224*3(3^2+64) = 10988544 \ll 86704128$$

Hiệu quả có thể thấy ngay, giảm gấp $\approx 7.9$ lần.

Việc tính tỉ lệ FLOPs khi áp dụng DSC so với conv được tính theo công thức sau:
$$r_{FLOPs} = \frac{H*W*M(K^2+N)}{H*W*K^2*M*N} = \frac{K^2+N}{K^2*N} = \frac{1}{K^2} + \frac{1}{N}$$

Số parameter của DSC:
$$\text{layer param} = (K*K)*M + (1*1*M*N) = M*(K^2+N)$$
Lại lấy ví dụ với $K=3, M=128, N=256$, ta đã có:
$$\text{layer param} = 128*(3^2+256) = 33920 \ll 294912$$

Trong ví dụ này, ta có thể thấy số param giảm đến $\approx 8.7$ lần!

Ta cũng tính tỉ lệ số param của DSC so với conv:
$$r_{param} = \frac{M*(K^2+N)}{K^2*M*N} =  \frac{1}{K^2} + \frac{1}{N}$$

Như vậy, cả FLOPs và số param đều có cùng 1 tỉ lệ giảm. Tỉ lệ này không hề phụ thuộc vào kích thước đầu vào, tạo sự ổn định khi huấn luyện cũng như chạy thực nghiệm.

Bây giờ, ta sẽ đi qua các thế hệ của MobileNet.

# 4. MobileNetv1
Là phiên bản đầu tiên sử dụng DSC để giảm mạnh số lượng param cũng như FLOPs so với phép chuẩn.

Câu trúc tổng thể:
1 conv(3x3, stride=2, out_channel=32) -> BN + ReLU -> 14 Block DSC -> Global average pooling -> FC.

Trong cấu trúc, sẽ có 1 số khối DSC có stride = 2 để giảm kích thước.

Ngoài ra, Google còn giới thiệu thêm 2 tham số giúp model trở nên linh động hơn nữa:
- $\alpha$: là tham số giảm số kênh trong các khối DSC(max = 1), giúp giảm bớt gánh nặng tính toán. Nên sử dụng trong khoảng [0.5, 1]
    
    VD: Nếu $\alpha = 0.75$, thì các out_channel trong các khối DSC sẽ tự động giảm còn 75%.
- $\rho$: là tham số điều chỉnh kích thước đầu vào để tăng tốc độ tính.

    VD: Nếu $\rho = 128$ thì ảnh sẽ được resize xuống còn 128 x 128 trước khi đưa vào architecture.

# 5. MobileNetv2
Là bản cải tiến mạnh từ v1, vẫn giữ cốt lõi nhanh và gọn. Được cải tiến từ phát hiện rằng MobileNetv1 khi xuống càng sâu, sự mất mát thông tin ngày càng lớn. Vì thế, lần cải tiến này đã đưa ra 2 cấu trúc mới:
- Inverted residual block(IRB): Khối này đã kết hợp 2 yếu tố:
    - Inverted: Trong ResNet, khối BottleNeck (dành cho architecture từ 50+ lớp) sẽ được thi triển theo kiểu: Nén -> Xử lý -> Giải nén. Mục đích là để giảm số lượng kênh trước khi vào conv 3x3. Ở v2, người ta cải tiến khối DSC của v1, thêm 1 layer conv 1x1 ở trước nó để tăng kích thước, vì thế thứ tự mới sẽ là: Mở rộng -> Xử lý -> nén lại (bottleneck).

    <img src="image1.png">
     
    - Skip connect: Để xử lý vấn đề hao hụt thông tin do tác dụng phụ DSC, 1 skip connection đã được thêm vào như trong ResNet để bảo toàn thông tin.
- Linear bottleneck: Đây chính là phát hiện quan trọng giúp v2 đạt được chính xác cao hơn nữa.
    - Khi dữ liệu được cô đọng xuống 1 chiều không gian nhỏ, việc áp dụng các hàm phi tuyến sẽ cắt bỏ đi các giá trị âm, làm giảm đi vùng giá trị có ý nghĩa.
    - Vì thế, trong bộ phận nén lại của khối IRB, người ta không áp dụng thêm bất cứ lớp phi tuyến nào cả(linear).

Cấu trúc tổng thể: conv(3x3, stride=2)-> IRB x 17 -> 1x1 conv -> Global average pool -> FC

# 6. MobileNetv3

Đây là bản cải tiến lớn từ v2 khi 3 yếu tố mới được giới thiệu:
- Squeeze and excitation (SE Block): Nhận thấy không hẳn lớp nào cũng có tầm quan trọng tương đương. Để model dễ tập trung vào thông tin quan trọng hơn, người ta thêm có chế layer-attention như sau:
    - Squeeze (Nén):Dùng GAP để nén tất cả thông tin của các kênh, tạo ra 1 vector tóm tắt trọng số của mỗi kênh.
    - Excitation (Kích thích/đánh trọng số):Sử dụng 2 lớp FC nhỏ để học mối quan hệ giữa các kênh và tạo ra 1 tập các trọng số (attention score), sau đó áp dụng chúng lại cho từng kênh ban đầu.Điều này giúp model chú ý đến các kênh quan trọng và giảm ảnh hưởng các kênh ít quan trọng hơn.

        <img src="image2.png" width="900">

- H-swish (hard-swish) activation: Với nhận định rằng những hàm ReLU hay ReLU6 chỉ cắt đi phần âm, giảm đi độ tinh xảo của các tensor, 1 non-linear activation mới được giới thiệu, là bản cải tiến của cả hàm swish lẫn ReLU6:
$$\text{h-swish}(x) = x * ReLU6(x+3)/6$$
Hàm phi tuyến này đã được chứng minh là nhẹ hơn swish vốn dùng sigmoid, đồng thời tăng độ chính xác cuối cùng.
- Dùng AutoML để tìm kiến trúc tốt nhất: Google đã dùng NAS(Neural architecture search) để lựa chọn:
    - số block, số kênh
    - kernel_size, stride và loại nonlinear cho từng block
    - việc có sử dụng SE cho từng block hay không

Sau khi tìm kiếm xong, họ khóa kiến trúc lại thành 2 phiên bản: MobileNetv3-Large và MobileNetv3-Small.


# 7. MobileNetv4
Là bản mới nhất, mạnh nhất. Được công bố vào thàng 4/2024, đây là 1 bước nhảy vọt không chỉ về hiệu suất, mà còn nhờ cách nó có thể được tối ưu hóa hiệu suất trên nhiều loại phần cứng di động khác nhau. Có 3 sự thay đổi chính:
- Universal inverted bottleneck (UIB block): Thay vì chỉ dựa vào 1 loại bottleneck cố định, UIB kết hợp và tổng hợp các ý tưởng tốt nhất từ các architecture trước:
    - Inverted bottleneck của v2: Vẫn làm theo phương thức mở rộng -> xử lý bằng Depthwise -> nén lại.
    - Các ý tưởng từ ConvNeXt: TLà phiên bản mạnh nhất của CNN, các ý tưởng đã được tiếp thu bao gồm:
        - Sử dụng các kernel size lớn hơn cho Depthwise convolution: ConvNeXt đã chứng minh được việc sử dụng các kernel kích thước lớn (7x7, 9x9) ngay cả trong các lớp sâu đã giúp các lớp conv có tầm nhìn rộng hơn trong không gian cục bộ, mô phỏng lại được khả năng nhìn toàn cục self-attention của Transformer.
        - Khối FFN (feed-forward network) được tích hợp: Các mô hình Transformer sử dụng 1 khối FFN bao gồm 2 lớp tuyến tính với 1 lớp phi tuyến ở giữa(vd: Linear -> GELU -> Linear) để tăng khả năng biến đổi và biểu diễn của mô hình. V4 đã đưa 1 khối tương tự FFN vào sau khối UIB để mô phỏng khả năng xử lý đặc trưng của Transformer.
        - Thay đổi thứ tự chuẩn hóa: v4 đã thử nghiệm với việc thay đổi thứ tự của các lớp và việc sử dụng chuẩn hóa như ConvNeXt.
- Mobile MQA (Mobile multi-query attention): v4 đã tối ưu hóa MQA để phù hợp với kiến trúc bộ nhớ và tính toán của các chip di động. Điều này giúp đẩy nhanh tốc độ xử lý các phép attention lên đáng kể(tăng 39% trên Pixel 8 edgeTPU).
- Tối ưu hóa quy trình NAS:
    - Google đã phát triển 1 quy trình NAS tinh vi hơn, cho phép tìm kiếm kiến trúc hiệu quả và chính xác hơn.
    - Khả năng tìm các mô hình là Pareto optimal - là tối ưu về cả độ chính xác và tốc độ trên nhiều nền tảng phần cứng khác nhau.
    - Kĩ thuật distillation mới: Google sử dụng 1 kĩ thuật distillation mới để các mô hình v4 học hiệu quả hơn từ các mô hình teacher, kết hợp dữ liệu và các kĩ thuật data augmentation đa dạng giúp đạt được acc cao hơn.

Riêng phần này thì cài tay không nổi, vì quá nhiều thành phần mới. Bản implementation official của Google: https://github.com/tensorflow/models/blob/master/official/vision/modeling/backbones/mobilenet.py


# 8. Các biến thể:
Ngoài các phiên bản chính, còn có các bản biến thể được điều chỉnh:
- Cho thiết bị biên: MobileNetV3EdgeTPU, ...
- Đa phương thức: MobileNetMultiMAX, MobileNetMultiMAXSeg,...

# 9. Kết luận: 
Dòng MobileNet mang lại lựa chọn hiệu quả hơn rất nhiều cho các thiết bị tài nguyên thấp, đông thời cũng là cơ sở của nhiều hướng đi và lý thuyết mới.